# Exploracion de datos 

# Importacion de datos desde API
No requiere credenciales

In [ ]:
from typing import Final
import requests
import zipfile
import io
import pandas as pd
import os
import logging
import warnings
import matplotlib.pyplot as plt


warnings.filterwarnings('ignore')

DATA_DIR: Final = os.path.join(
    os.path.dirname(os.getcwd()),
    'data',
    'raw'
)

file_path = os.path.join(DATA_DIR, 'online_retail_II.xlsx')

if not os.path.exists(file_path):
    url = "https://archive.ics.uci.edu/static/public/502/online+retail+ii.zip"
    response = requests.request("GET", url)
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))
    zip_file.extractall(path=DATA_DIR)
    logging.info("Downloaded and extracted the zip file")

# Load the .xlsx file into a pandas dataframe
year_2009_2010 = pd.read_excel(file_path, sheet_name='Year 2009-2010')
year_2010_2011 = pd.read_excel(file_path, sheet_name='Year 2010-2011')
logging.info("Loaded the data into a pandas dataframe")

df = pd.concat([year_2009_2010, year_2010_2011], ignore_index=True)
df.head(1)


: 

Observamos los tipos de datos disponibles

In [ ]:
df.dtypes

: 

In [ ]:
df.shape

: 

In [ ]:
df.isnull().sum()

: 

# Descripción de las variables

In [ ]:
# estadísticas de las columnas categoricas
df.describe( include=['object', 'string'])

: 

Podemos ver que en el campo Description tenemos valores nulos a analizar. El país con mayores ventas es United Kingdom, tenemos 40 países. El Invoice es el indicador de factura, en realidad en todo el dataset se realizaron 28816 ventas facturadas. 

In [ ]:
df.describe( include=['float64', 'int64'])

: 

En la variable Quantity hay valores fuera de serie grandes, los datos se concentran en valores cercanos a 10, por lo que existe un sesgo a la derecha, es decir los valores tienen una asímetría positiva. Por ende, los datos deben ser escalados o aplicados con un logaritmo en caso de aplicar modelos estadísticos con supuestos en distribuciones normales. El precio presenta el mismo problema de asimetría positiva. Para el Customer se debe cambiar de número a objeto ya que nos interesa la cantidad que veces que compró un mismo cliente, no operaciones entre el ID. 

In [ ]:
df.describe( include= 'datetime64[ns]')

: 

En cuanto al tiempo, el rango o periodo es desde 2009 diciembre hasta 2011 diciembre, abarcando un dos años de ventas online. 

# Analisis de valores faltantes

Lo que realizaremos es un analisis de los valores faltantes para descartar cualquier valor nulo por error de digitación. Para ello, compararemos por StockCode para ver si en el periodo de estudio existe un misdo StockCode con el valor que podría ir en los valores faltantes. 

In [ ]:
# Step 1: Filter rows where Description or CustomerID is missing
missing_values_df = df[df['Description'].isna() | df['Customer ID'].isna()]

# Step 2: Group by Stockcode or InvoiceID
grouped_by_stockcode = missing_values_df.groupby('StockCode')

# Stockcode example
inconsistent_stockcode = grouped_by_stockcode.apply(
    lambda x: x[['Description', 'Customer ID']].isna().all(axis=0)
).reset_index()

# Filtering the inconsistent groups where not all values are missing
inconsistent_stockcodes = inconsistent_stockcode[inconsistent_stockcode.any(axis=1)]

: 

In [ ]:
# no siempre son valores nulos estos codigos:
inconsistent_stockcodes.loc[inconsistent_stockcodes.Description == False]

: 

Como ejemplo, veamos gift_0001_80 de codigo de Stock, este tiene dos valores posibles como vemos a continuación:

In [ ]:
df.loc[df['StockCode'] == 'gift_0001_80'].Description.unique() 

: 

Como vimos, hay valores que tienen dos opciones o hasta 3. Por ahora vamos a cambiar los valores de nan por el valor que se utiliza en la serie:

In [ ]:
# Step 1: Group by StockCode and get unique values in Description
unique_values = df.groupby('StockCode')['Description'].unique()

# Step 2: Identify StockCodes with exactly one non-null and one NaN value
stockcodes_to_impute = unique_values[unique_values.apply(lambda x: len(x) == 2 and pd.isna(x).any())]

# Step 3: Create a dictionary for imputation (mapping StockCode to its non-null Description)
imputation_dict = {stockcode: next(val for val in values if pd.notna(val)) 
                   for stockcode, values in stockcodes_to_impute.items()}


: 

Acá vemos una lista de los códgios de Stock y su respectivo valor no nulo

In [ ]:
imputation_dict

: 

Vamos a proceder a remplazar los valores nulos cuando al descripción es un valor nulo y su codigo está en el diccionario que creamos:

In [ ]:
# Step 4: Apply the imputation only for the same StockCode
df['Description'] = df.apply(
    lambda row: imputation_dict[row['StockCode']] 
    if pd.isna(row['Description']) and row['StockCode'] in imputation_dict else row['Description'], axis=1
)

: 

Veamos el mismo ejemplo de antes:

In [ ]:
df.loc[df['StockCode'] == 'gift_0001_80'].Description.unique() 

: 

Procedemos a ver los valores nulos nuevamente:

In [ ]:
df.isna().sum()

: 

reducimos los valores nulos de Description de 4382 a 1320 una reducción del 70% aproximadamente con este enfoque.
Ahora veamos los valores faltantes y detectemos cuando estos valores son solamente nulos:

In [ ]:
# Step 1: Filter rows where Description or CustomerID is missing
missing_values_df = df[df['Description'].isna() | df['Customer ID'].isna()]

# Step 2: Group by Stockcode or InvoiceID
grouped_by_stockcode = missing_values_df.groupby('StockCode')

# Stockcode example
inconsistent_stockcode = grouped_by_stockcode.apply(
    lambda x: x[['Description', 'Customer ID']].isna().all(axis=0)
).reset_index()

# Filtering the inconsistent groups where not all values are missing
inconsistent_stockcodes = inconsistent_stockcode[inconsistent_stockcode.any(axis=1)]



: 

In [ ]:
# no siempre son valores nulos estos codigos:
inconsistent_stockcodes.loc[inconsistent_stockcodes.Description == True]

: 

In [ ]:
df.loc[df['StockCode'] == 'gift_0001_60'].Description.unique() 

: 

Como vemos, tenemos 361 casos en donde el valor de Descripcion siempre es nulo. Ahora para analizar como la eliminación de estos valores afectaría un modelo predictivo: 

In [ ]:
# agrupamos por StockCode
grouped_by_stockcode = df.groupby('StockCode')
stockcodes_with_nan = grouped_by_stockcode.apply(lambda x: x['Description'].isna().all())
stockcodes_with_nan = stockcodes_with_nan[stockcodes_with_nan].index.tolist()

: 

In [ ]:
stockcodes_with_nan
# un vistazo de los StockCode que siempre son nulos en descripcion

: 

Podemos ver un resumen estadistico de los valores que siempre son nulos en StockCode:

In [ ]:
df.loc[df['StockCode'].isin(stockcodes_with_nan)].describe(include=['int64', 'float64'])

: 

Con esto vemos que también son siempre nulos los valores de Customer ID, el precio es 0 siempre y las cantidades son negativas o con un maximo de 160 cantidades, concentrandoce en -44 los datos, pero tenemos una asímetría positiva en los datos, es decir hay valores muy negativos que afectan el calculo llegando hasta -5000 aproximadamente. 


Por lo anterior, podemos eliminar los valores nulos, abajo dejo una grafica donde muestro que con esto se afecta únicamente los valores de reino unido. 

In [ ]:
df.loc[df['StockCode'].isin(stockcodes_with_nan)]['Country'].hist()
plt.show()

: 

In [ ]:
df = df[~df['StockCode'].isin(stockcodes_with_nan)]

: 

In [ ]:
df.describe()

: 